# Extract coastal waterlines across time
**What does this notebook do?** 

This notebooks demonstrates how to tidally tag remotely sensed imagery using the [OSU Tidal Prediction Software or OTPS](http://volkov.oce.orst.edu/tides/otps.html) model, create geomedian composites for a given set of epochs and tidal range, and extract waterline contours from the composite layers for each epoch. 

**Requirements:** 

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea/20180515`  *(currently using an older version of `dea` due to a bug in `xr.concat`; will be reverted to `module load dea` in future)*

`module load otps`

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

**Date:** September 2018

**Author:** Robbi Bishop-Taylor

## Import modules

In [ ]:
import os
import sys
import datacube
import itertools
import warnings
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from otps import TimePoint
from otps import predict_tide
from datetime import datetime, timedelta
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from datacube_stats.statistics import GeoMedian
from dateutil.relativedelta import relativedelta
from datacube.helpers import ga_pq_fuser
from datacube.helpers import write_geotiff
from datacube.storage import masking
from shapely.geometry import Point
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

sys.path.append('../10_Scripts')
import SpatialTools

# For nicer notebook plotting, hide warnings (comment out for real analysis)
warnings.filterwarnings('ignore') 

# Create datacube instance
dc = datacube.Datacube(app='Tidal geomedian filmstrips')

def date_range(start_date, end_date, increment, period):
    
    """Generate dates seperated by given time increment/period"""
    
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

def ds_to_rgb(ds, bands=['red', 'green', 'blue'], reflect_stand=4000):
    
    """Converts an xarray dataset to a three band array"""
    
    rawimg = np.transpose(ds[bands].to_array().data, [1, 2, 0])
    img_toshow = (rawimg / reflect_stand).clip(0, 1)
    return img_toshow


In [8]:

from datacube import Datacube
from datacube.model import Range
from datetime import datetime
ARDremotedc = Datacube(config='/home/561/rt1527/.ard-interoperability.conf')
 
date_1= datetime(2016, 8, 1, 0, 0)
date_2= datetime(2016, 8, 17, 0, 0)
x1=140.22
x2=145.23
y1=-34.58
y2=-34.59
query = {
         'time': (date_1,date_2),
         'x' : (x1, x2),
         'y' : (y1, y2),
         'output_crs' : 'EPSG:4326',
         'resolution': (-0.0001,0.0001)
           }
data = ARDremotedc.load(product='ls8_ard',  **query)
data

LookupError: ('Unknown arguments: ', {'time'})

In [7]:
import datetime as dt
import pandas as pd
import numpy as np

# Example data
dates = pd.date_range('2011-01-01', periods=25, freq='W')
dates = dates.append(dates[10:20])  # Create duplicates
df = pd.DataFrame({'values': np.random.randn(len(dates)),
                   'date': dates}, index=range(len(dates)))
print(df)

# Example date
DateToExtract = dt.datetime(2012, 1, 1)

# Row that has the smallest absolute difference (i.e. either before
# or after the DateToExtract is the closest; get index
closest_index = abs(df.date - DateToExtract).idxmin()

# Subset
df.iloc[[closest_index]]

      values       date
0   1.490608 2011-01-02
1   0.716810 2011-01-09
2   0.124855 2011-01-16
3  -0.402116 2011-01-23
4   0.054888 2011-01-30
5  -0.778704 2011-02-06
6   0.563152 2011-02-13
7  -1.343159 2011-02-20
8   0.244016 2011-02-27
9  -0.803574 2011-03-06
10 -1.215627 2011-03-13
11 -2.288238 2011-03-20
12 -0.449190 2011-03-27
13 -0.280904 2011-04-03
14  0.205058 2011-04-10
15  0.157664 2011-04-17
16  0.889344 2011-04-24
17  1.138114 2011-05-01
18 -0.249259 2011-05-08
19  0.120709 2011-05-15
20  1.371686 2011-05-22
21  0.040607 2011-05-29
22  1.177085 2011-06-05
23 -0.386156 2011-06-12
24 -0.649618 2011-06-19
25  1.167408 2011-03-13
26  0.338236 2011-03-20
27  0.887707 2011-03-27
28  1.513982 2011-04-03
29  1.492196 2011-04-10
30  0.030728 2011-04-17
31 -1.296572 2011-04-24
32 -0.781270 2011-05-01
33  0.471042 2011-05-08
34  0.447963 2011-05-15


,values,date
24,-0.649618,2011-06-19


## Geomedian filmstrip parameters
Set the area, time period  and sensors of interest, and tide limits and epoch length used to produce each geomedian composite. This is the only cell that needs to be edited to run the notebook.

In [ ]:
# # Set up centre of study area and buffer size in metres for data extraction
# study_area = 'pointstuart_wide'  # name used as prefix for output files
# lat, lon = -12.238, 131.849  # centre of study area
# buffer = 15000  # metre units to extend region of interest on each side of centre point

# # Set up centre of study area and buffer size in metres for data extraction
# study_area = 'mitchell'  # name used as prefix for output files
# lat, lon = -15.14995, 141.63650  # centre of study area
# buffer = 10000  # metre units to extend region of interest on each side of centre point

# # Set up centre of study area and buffer size in metres for data extraction
# study_area = 'inskip'  # name used as prefix for output files
# lat, lon = -25.7909598625, 153.073231901  # centre of study area
# buffer = 8000  # metre units to extend region of interest on each side of centre point

# # Set up centre of study area and buffer size in metres for data extraction
# study_area = 'narrabeen'  # name used as prefix for output files
# lat, lon = -33.71773611412014, 151.30049228668213  # centre of study area
# buffer = 5000  # metre units to extend region of interest on each side of centre point

# # Set up centre of study area and buffer size in metres for data extraction
# study_area = 'victoriariver_2yr'  # name used as prefix for output files
# lat, lon = -15.4440342949, 129.811912341  # centre of study area
# buffer = 8000  # metre units to extend region of interest on each side of centre point

# # Set up centre of study area and buffer size in metres for data extraction
# study_area = 'victoriariver2_2yr'  # name used as prefix for output files
# lat, lon = -15.4475061785, 130.128534164  # centre of study area
# buffer = 8000  # metre units to extend region of interest on each side of centre point

# Set up centre of study area and buffer size in metres for data extraction
study_area = 'ard_test'  # name used as prefix for output files
lat, lon = -19.4947481277, 147.404945988  # centre of study area
buffer = 20000  # metre units to extend region of interest on each side of centre point

# Set up compositing parameters
time_period = ('2015-01-01', '2019-01-01')  # Total date range to import data from
sensors = ['ls5', 'ls7', 'ls8']  # Landsat sensors from which to import data from
ls7_slc_off = False  # Whether to include striped > May 31 2003 Landsat 7 SLC-off data
lower_tideheight = 0.666 # Minimum proportion of the observed tidal range to include 
upper_tideheight = 1.0  # Maximum proportion of the observed tidal range to include 
epoch_years = 1  # Length of each epoch used to compute geomedians (i.e. 5 years = 1988 to 1993)


In [ ]:
# sensor_all = dc.load(product = 'ls8_nbart_albers', 
#                  group_by = 'solar_day', 
#                  measurements = ['green', 'nir', 'swir1', 'swir2'],
#                  **query)


dc.list_measurements()

## Return observations for each sensor for the entire time series
Use `dask` to lazily load Landsat data and PQ for each sensor for the entire time series. No data is actually loaded here: this is saved until after the layers have been filtered by tidal stage and date to save processing time.

In [ ]:
# If output data and figure directories doesn't exist, create them
if not os.path.isdir('output_data/{}/'.format(study_area)):
    os.makedirs('output_data/{}/'.format(study_area))
    
if not os.path.isdir('figures/{}/'.format(study_area)):
    os.makedirs('figures/{}/'.format(study_area))

# Create list of epochs between start and end of time_period in datetime format
start = datetime.strptime(time_period[0], '%Y-%m-%d')
end = datetime.strptime(time_period[1], '%Y-%m-%d')
epochs = date_range(start, end, epoch_years, 'years')

# Print list of epochs
epochs_strings = [epoch.strftime('%Y-%m-%d') for epoch in epochs][:-1]
print('Processing {} epochs: {}'.format(len(epochs_strings), ', '.join(epochs_strings)))

# Identify tide point by importing ITEM polygons
polygons_gpd = gpd.read_file('/g/data/r78/intertidal/GA_native_tidal_model.shp')
polygon_gpd = polygons_gpd[polygons_gpd.intersects(Point(lon, lat))]
tidepost_lon, tidepost_lat = polygon_gpd[['lon', 'lat']].values[0]
# tidepost_lon, tidepost_lat = 129.52, -14.89
print('Using tidepost coordinates: {} E, {} S\n'.format(tidepost_lon, tidepost_lat))

# Set up query
x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {'x': (x - buffer, x + buffer),
         'y': (y - buffer, y + buffer),
         'time': time_period,
         'crs': 'EPSG:3577'}

# Output dicts to hold entire time-series for each sensor
sensor_dict = {}
pq_dict = {}

# For each sensor, dask load data and compute tide heights for each sensor
for sensor in sensors:
    
    # The following code will fail if there is no data for the sensor in time_period;
    # this try statement catches this and skips the affected sensor    
    try:
    
        # Return observations matching query without actually loading them using dask
        sensor_all = dc.load(product = '{}_nbart_albers'.format(sensor), 
                         group_by = 'solar_day', 
                         measurements = ['green', 'nir', 'swir1', 'swir2'],
                         dask_chunks={'time': 1},
                         **query)

        # Load PQ data matching query without actually loading them using dask
        pq_all = dc.load(product = '{}_pq_albers'.format(sensor),
                        group_by = 'solar_day',
                        fuse_func=ga_pq_fuser, 
                        dask_chunks={'time': 1},
                        **query)
        
        # Remove Landsat 7 SLC-off from PQ layer if ls7_slc_off = False
        if not ls7_slc_off and sensor == 'ls7':
            sensor_all = sensor_all.where(sensor_all.time < np.datetime64('2003-05-30'), drop=True) 

        # Use the tidal mode to compute tide heights for each observation:
        print('Computing tidal heights for {} {} observations'.format(len(sensor_all.time), sensor))
        obs_datetimes = sensor_all.time.data.astype('M8[s]').astype('O').tolist()
        obs_timepoints = [TimePoint(tidepost_lon, tidepost_lat, dt) for dt in obs_datetimes]
        obs_predictedtides = predict_tide(obs_timepoints)
        obs_tideheights = [predictedtide.tide_m for predictedtide in obs_predictedtides]

        # Assign tide heights to the dataset as a new variable
        sensor_all['tide_heights'] = xr.DataArray(obs_tideheights, [('time', sensor_all.time)]) 

        # Append results for each sensor to a dictionary with sensor name as the key
        sensor_dict[sensor] = sensor_all
        pq_dict[sensor] = pq_all 
    
    except AttributeError:
        
        print('{} selection failed due to likely lack of data in time_period; skipping'.format(sensor))

## Subset data by tidal height and epoch time range
Use image time-stamps to compute tidal heights at the time of each observation, and take a subset of the entire Landsat and PQ time series based on tide height and epoch date range for each sensor.

In [ ]:
# If any sensor has 0 observations, remove it from the dictionary before proceeding
sensor_dict = {key:value for key, value in sensor_dict.items() if len(value.time) > 0}

# Calculate max and min tide heights for the entire time series and all sensors
obs_min = np.min([sensor_ds.tide_heights.min() for sensor_ds in sensor_dict.values()])
obs_max = np.max([sensor_ds.tide_heights.max() for sensor_ds in sensor_dict.values()])
obs_range = obs_max - obs_min

# Calculate tidal limits used for subsequent data selection
sel_min = obs_min + (obs_range * lower_tideheight)
sel_max = obs_min + (obs_range * upper_tideheight)
print('Keeping tidal heights of {0:.2f} m to {1:.2f} m out of an observed local tidal ' 
      'range of {2:.2f} m to {3:.2f} m'.format(sel_min, sel_max, obs_min, obs_max))

# Create dictionaries to hold filtered sensor data for each epoch
sensor_epoch_dict = defaultdict(list)
pq_epoch_dict = defaultdict(list)
   
for epoch, sensor in itertools.product(epochs[:-1], sensor_dict.keys()):
                                           
    # Select dataset
    sensor_all = sensor_dict[sensor]  
    pq_all = pq_dict[sensor] 
    
    # Filter to keep only observations that have matching PQ data 
    time = (sensor_all.time - pq_all.time).time
    sensor_subset = sensor_all.sel(time=time)
    pq_subset = pq_all.sel(time=time)
                                           
    # Filter by tidal stage
    sensor_subset = sensor_subset.where((sensor_subset.tide_heights >= sel_min) &
                                        (sensor_subset.tide_heights <= sel_max), drop = True)
    
    # Filter pq to same timesteps (this avoids conversion to float by `.where`)
    pq_subset = pq_subset.sel(time = sensor_subset.time)

    # Identify from and to date strings
    from_date = epoch.strftime('%Y-%m-%d')
    to_date = (epoch + relativedelta(years=epoch_years)).strftime('%Y-%m-%d')
    
    # Subset sensor to from and to date for epoch
    print('    Filtering from {} to {} for {}'.format(from_date, to_date, sensor))
    sensor_subset = sensor_subset.sel(time=slice(from_date, to_date))
    pq_subset = pq_subset.sel(time=slice(from_date, to_date))

    # Add subsetted data to dicts (one key matching a list of sensor data for each epoch)
    sensor_epoch_dict[from_date].append(sensor_subset)
    pq_epoch_dict[from_date].append(pq_subset)
    

## Combine multiple sensors, load data and generate geomedians
For each epoch, combine all sensors into one dataset, load the data for the first time using `dask`'s `.compute()`, then composite all timesteps into a single array using a geometric median computation.

In [ ]:
# Dict to hold output geomedian composits
ndwi_dict = {}
mndwi_dict = {}
awei_dict = {}
tideheights_dict = {}

for from_date in sensor_epoch_dict.keys():
    
    # For the first time, actually load data for all sensors and then combine into one dataset  
    print('Loading and combining {} data for {}'.format(", ".join(sensor_dict.keys()), from_date))
    sensor_combined = xr.concat([i.compute() for i in sensor_epoch_dict[from_date]], dim='time')
    pq_combined = xr.concat([i.compute() for i in pq_epoch_dict[from_date]], dim='time')
    print('    {} observations combined'.format(len(sensor_combined.time)))
    
    # Manually add flag definition back in (it is lost during the concatenation)
    pq_combined.pixelquality.attrs['flags_definition'] = pq_all.pixelquality.flags_definition
    
    # Sort output datasets by date
    sensor_combined = sensor_combined.sortby('time')
    pq_combined = pq_combined.sortby('time')
    
    # Identify pixels with no clouds/shadows in either ACCA for Fmask
    good_quality = masking.make_mask(pq_combined.pixelquality,
                                     cloud_acca='no_cloud',
                                     cloud_shadow_acca='no_cloud_shadow',
                                     cloud_shadow_fmask='no_cloud_shadow',
                                     cloud_fmask='no_cloud',
                                     blue_saturated=False,
                                     green_saturated=False,
                                     red_saturated=False,
                                     nir_saturated=False,
                                     swir1_saturated=False,
                                     swir2_saturated=False,
                                     contiguous=True)
    
    # Apply mask to set all PQ-affected pixels to NaN and set nodata to NaN
    print('    Applying PQ mask and setting nodata to NaN')
    sensor_combined = sensor_combined.where(good_quality)
    sensor_combined = masking.mask_invalid_data(sensor_combined)
    
    # Compute NDWI
    sensor_combined["ndwi"] = (sensor_combined.green - sensor_combined.nir) / (sensor_combined.green + sensor_combined.nir)
    sensor_combined["mndwi"] = (sensor_combined.green - sensor_combined.swir1) / (sensor_combined.green + sensor_combined.swir1)
    sensor_combined["awei"] = (4 * (sensor_combined.green * 0.0001 - sensor_combined.swir1 * 0.0001) -
                              (0.25 * sensor_combined.nir * 0.0001 + 2.75 * sensor_combined.swir2 * 0.0001))

    # Compute NDWI composite using all timesteps
    print('    Computing NDWI, MNDWI and AWEI median')
    ndwi_median = sensor_combined[["ndwi"]].median(dim='time', keep_attrs=True)
    mndwi_median = sensor_combined[["mndwi"]].median(dim='time', keep_attrs=True)
    awei_median = sensor_combined[["awei"]].median(dim='time', keep_attrs=True)
    
    # Export to file
    print('    Exporting to file')
    filename_ndwi = 'output_data/{0}/{0}_{1}_ndwi.tif'.format(study_area, from_date)
    filename_mndwi = 'output_data/{0}/{0}_{1}_mndwi.tif'.format(study_area, from_date)
    filename_awei = 'output_data/{0}/{0}_{1}_awei.tif'.format(study_area, from_date)
    write_geotiff(filename=filename_ndwi, dataset=ndwi_median)
    write_geotiff(filename=filename_mndwi, dataset=mndwi_median)
    write_geotiff(filename=filename_awei, dataset=awei_median)
    
    # Assign to dict
    ndwi_dict[from_date] = ndwi_median
    mndwi_dict[from_date] = mndwi_median
    awei_dict[from_date] = awei_median
    

## Extract waterlines

In [ ]:
threshold = 0
metric = 'ndwi'

for date, data in ndwi_dict.items():
    
    print(date)
    
    # Prepare attributes as input to contour extract
    attribute_data = {'date': [date[0:4]]}
    attribute_dtypes = {'date': 'int'}
    
    # Extract contours with custom attribute fields:
    contour_dict = SpatialTools.contour_extract(z_values=[threshold],
                                   ds_array=data[metric],
                                   ds_crs='epsg:3577',
                                   ds_affine=data.geobox.transform,
                                   output_shp=f'output_data/{study_area}/{study_area}_{metric}_{date}.shp',
                                   attribute_data=attribute_data,
                                   attribute_dtypes=attribute_dtypes)
    
# Combine all shapefiles into one file
import glob
shapefiles = glob.glob(f'output_data/{study_area}/{study_area}_{metric}_*01-01.shp')
gdf = pd.concat([gpd.read_file(shp) for shp in shapefiles], sort=False).pipe(gpd.GeoDataFrame)

# Save as combined shapefile
gdf = gdf.reset_index()[['date', 'geometry']].sort_values('date')
gdf['date'] = gdf['date'].astype(np.float32)
gdf.crs = 'epsg:3577'
gdf.to_file(f'output_data/{study_area}/{study_area}_{metric}_combined.shp')

gdf['date'].dtype


## Create tidal modelled vs observed plot
Create a plot comparing selected Landsat observations to all Landsat observations and the entire tidal history of the study area:

In [ ]:
# For each hour between start and end of time series, predict tide and add to list
all_times = date_range(start, end, 1, 'hours')
tp_model = [TimePoint(tidepost_lon, tidepost_lat, dt) for dt in all_times]
tides_model = [tide.tide_m for tide in predict_tide(tp_model)]

# Covert to dataframe of modelled dates and tidal heights
modelled_df = pd.DataFrame({'tide_heights': tides_model}, index=pd.DatetimeIndex(all_times))

# Return dataframe of previously-generated observed dates and tidal heights
observed_df = xr.concat([i.tide_heights for i in sensor_dict.values()], dim='time').to_dataframe()   

# Set up plot
fig = plt.figure(figsize=(10, 4))
plt.margins(0)
fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)
fig.axes[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.ylabel('Tide height (m)')

plt.annotate('Tidepost: {} E, {} S ({})'.format(tidepost_lon, tidepost_lat, study_area), 
             xy=(0, 1.2), xycoords='axes fraction', 
             xytext=(5, -3), textcoords='offset points',
             fontsize=10, verticalalignment='top', fontweight='bold') 

# Plot modelled values as grey background
plt.plot(modelled_df.index, modelled_df.tide_heights,
         color='gainsboro', linewidth=0.6, zorder=1, label='OTPS model')

# Plot all observations as black points
plt.scatter(observed_df.index, observed_df.tide_heights,
            s=4, color='darkgrey', marker='o', zorder=2, label='All observations')

# Plot selected observations in red by clipping observed_df to min and max tide selection
selected_df = observed_df[observed_df.tide_heights.between(sel_min-0.001, sel_max+0.001)]
plt.scatter(selected_df.index, selected_df.tide_heights,
            s=10, color='black', marker='o', zorder=2, label='Selected observations')

# Plot horizontal lines defining border of selected tidal range
plt.axhline(y=sel_min, color='red', alpha=0.2) 
plt.axhline(y=sel_max, color='red', alpha=0.2) 

# Add vertical lines and annotation defining each epoch
for epoch in epochs[:-1]:
    
    # Compute from and to date strings
    from_date = epoch.strftime('%Y-%m-%d')
    to_date = (epoch + relativedelta(years=epoch_years)).strftime('%Y-%m-%d')
    
    # Add vertical line and epoch titles
    plt.axvline(x=epoch, color='red', alpha=0.2)
    plt.annotate('{} to {}'.format(from_date, to_date), 
                 xy=(epoch, modelled_df.tide_heights.max()), 
                 xytext=(5, 10), textcoords='offset points', fontsize=7) 
    
# Add legend
plt.legend(bbox_to_anchor=(0.975, 1.2), loc=1, borderaxespad=0, ncol=3, 
           handletextpad=0.1, frameon=False, columnspacing=0.3, fontsize=7)

# Export plot
filename = 'figures/{0}/{0}_tideobs.png'.format(study_area)
print('    Exporting to {}'.format(filename))
# fig.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0)